In [1]:
pip install torch torchvision nibabel numpy matplotlib scikit-image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 78.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.8.93
    Uninstalling nvidia-nvjitlink-cu12-12.8.93:
      Successfully uninstalled nvidia-nvjitlink-cu12-12.8.93
  Attempting uninstall: nvidia-curand-cu12
    Found existing installation: nvidia-curand-cu12 10.3.9.90
    Uninstalling nvidia-curand-cu12-10.3.9.90:
      Successfully uninstalled nvidia-curand-cu12-10.3.9.90
  Attemptin

In [2]:
pip install monai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 23.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import random
from glob import glob
import nibabel as nib
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from skimage.transform import resize

# -------------------------------------
# U-Net Architecture
# -------------------------------------
class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super().__init__()
        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
            )
        self.enc1 = conv_block(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.bottleneck = conv_block(128, 256)
        self.upconv2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = conv_block(256, 128)
        self.upconv1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = conv_block(128, 64)
        self.final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        b = self.bottleneck(self.pool2(e2))
        d2 = self.upconv2(b)
        d2 = self.dec2(torch.cat([d2, e2], dim=1))
        d1 = self.upconv1(d2)
        d1 = self.dec1(torch.cat([d1, e1], dim=1))
        return self.final(d1)

# -------------------------------------
# Slice Dataset (Load per Slice)
# -------------------------------------
class SliceDataset(Dataset):
    def __init__(self, slice_index_list, target_shape=(128, 128)):
        self.slice_index_list = slice_index_list
        self.target_shape = target_shape

    def __len__(self):
        return len(self.slice_index_list)

    def __getitem__(self, idx):
        img_path, mask_path, slice_idx, _ = self.slice_index_list[idx]

        img = nib.load(img_path).get_fdata()[:, :, slice_idx]
        mask = nib.load(mask_path).get_fdata()[:, :, slice_idx]

        img = resize(img, self.target_shape, preserve_range=True).astype(np.float32)
        mask = resize(mask, self.target_shape, preserve_range=True).astype(np.float32)

        if np.max(img) > 0:
            img = img / np.max(img)

        img = torch.tensor(img).unsqueeze(0)
        mask = torch.tensor((mask > 0).astype(np.float32)).unsqueeze(0)

        return img, mask

# -------------------------------------
# Build Slice Index List (Lazy Load)
# -------------------------------------
import os
import random
from glob import glob
import nibabel as nib
import numpy as np

def build_slice_index_list(images_dir, masks_dir, labeled_ratio=0.7):
    # Get all image and mask files
    image_files = glob(os.path.join(images_dir, "*.nii"))
    mask_files = glob(os.path.join(masks_dir, "*.nii"))

    # Create dictionaries keyed by patient ID (assuming filename format like 'patient123.nii.gz')
    def extract_id(filepath):
        return os.path.basename(filepath).split('.')[0]  # Adjust this based on your naming convention

    image_dict = {extract_id(p): p for p in image_files}
    mask_dict = {extract_id(p): p for p in mask_files}

    # Find intersection of IDs
    common_ids = set(image_dict.keys()) & set(mask_dict.keys())

    slice_metadata = []

    for pid in common_ids:
        img_path = image_dict[pid]
        mask_path = mask_dict[pid]

        mask_data = nib.load(mask_path).get_fdata()
        num_slices = mask_data.shape[2]

        for i in range(num_slices):
            has_label = np.any(mask_data[:, :, i] > 0)
            slice_metadata.append((img_path, mask_path, i, has_label))

    # Split slices into labeled/unlabeled
    labeled = [s for s in slice_metadata if s[3]]
    unlabeled = [s for s in slice_metadata if not s[3]]

    # Sample based on labeled ratio
    num_labeled = int(len(slice_metadata) * labeled_ratio)
    random.shuffle(labeled)
    random.shuffle(unlabeled)

    selected = labeled[:num_labeled] + unlabeled[:len(slice_metadata) - num_labeled]
    random.shuffle(selected)

    return selected

# -------------------------------------
# Dice Loss with Loss Mask
# -------------------------------------
def dice_loss(pred, target, loss_mask, smooth=1e-5):
    pred = torch.sigmoid(pred)
    intersection = (pred * target).sum()
    return 1 - ((2. * intersection + smooth) /
                ((pred * loss_mask).sum() + (target * loss_mask).sum() + smooth))

# -------------------------------------
# Training Loop
# -------------------------------------
import monai
seg_loss = monai.losses.DiceCELoss(sigmoid=True, squared_pred=True, reduction='mean')
import torch
# from torchmetrics import DiceLoss
from tqdm import tqdm

def train(model, train_dataloader,val_dataloader,test_dataloader, loss_fn, epochs=5, device='cuda'):
    model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
    best_dice=0
    # Initialize DiceLoss
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        # Wrap the dataloader with tqdm for a progress bar
        with tqdm(train_dataloader, desc=f"Epoch {epoch+1}/{epochs}", unit="batch") as pbar:
            for img, mask in pbar:
                img, mask = img.to(device), mask.to(device)

                img, mask = img.view(-1,1,128,128), mask.view(-1,1,128,128)
                # Forward pass
                out = model(img)

                # Calculate Dice loss using the DiceLoss function
                loss = loss_fn(out, mask)

                # Backpropagation
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                # Update total loss and progress bar description
                total_loss += loss.item()
                pbar.set_postfix(loss=loss.item())

        # After completing an epoch, print the average loss
        print(f"Epoch {epoch+1}: Loss = {total_loss / len(train_dataloader):.4f}")
        current_dice = evaluate(model, val_dataloader, 'cuda',best_dice)
        print("VALIDATION IoU = "+str(current_dice))
        if best_dice<current_dice:
            best_dice=current_dice
            torch.save(model.state_dict(),'best.pth')
    model.load_state_dict(torch.load('best.pth'))
    
    print("TEST IoU = "+str(evaluate(model, test_dataloader, 'cuda',best_dice)))


2025-05-09 14:04:10.872738: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746799451.098979      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746799451.165616      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
import random
import numpy as np
import torch
from torch.utils.data import Dataset
import nibabel as nib
from skimage.transform import resize
import re
import glob
        
def extract_number_from_path(path):
    match = re.search(r'(\d+)_image\.nii', path)
    if match:
        return match.group(1)
    else:
        return None  # Returns None if no match is found
    
class VolumeSliceDataset(Dataset):
    def __init__(self, images_dir, masks_dir, ids, target_shape=(128, 128)):
        # Dynamically load the image and mask paths using a list of IDs
        self.images_dir = images_dir
        self.masks_dir = masks_dir
        self.target_shape = target_shape
        
        # Store the IDs instead of full paths
        # Example usage:
        self.ids=ids
        # Ensure images and masks match
        
    def __len__(self):
        return len(self.ids)

    def __getitem__(self, idx):
        # Load the NIfTI file based on ID (idx)
        idn = self.ids[idx]
        
        img_path = self.images_dir+idn+"_image.nii"
        mask_path = self.masks_dir+idn+"_label.nii"
        
        # Load the image and mask data
        img_nii = nib.load(img_path)
        mask_nii = nib.load(mask_path)

        img_data = img_nii.get_fdata()
        mask_data = mask_nii.get_fdata()

        num_slices = img_data.shape[2]

        # Separate slices with and without ROI
        with_roi = []
        without_roi = []

        for i in range(num_slices):
            has_roi = np.any(mask_data[:, :, i] > 0)
            if has_roi:
                with_roi.append(i)
            else:
                without_roi.append(i)

        # 70% with ROI and 30% without
        num_total = min(16, num_slices)
        num_with = min(int(0.7 * num_total), len(with_roi))
        num_without = num_total - num_with

        # Shuffle the slices
        random.shuffle(with_roi)
        random.shuffle(without_roi)

        selected_slices = with_roi[:num_with] + without_roi[:num_without]
        random.shuffle(selected_slices)

        # Initialize lists to store 16 slices
        imgs = []
        masks = []

        for i in selected_slices:
            img = resize(img_data[:, :, i], self.target_shape, preserve_range=True).astype(np.float32)
            mask = resize(mask_data[:, :, i], self.target_shape, preserve_range=True).astype(np.float32)

            if np.max(img) > 0:
                img = img / np.max(img)

            # Convert to tensor and add to list
            img_tensor = torch.tensor(img).unsqueeze(0)  # [1, H, W]
            mask_tensor = torch.tensor((mask > 0).astype(np.float32)).unsqueeze(0)

            imgs.append(img_tensor)
            masks.append(mask_tensor)

        # Stack the slices to form a batch of 16 slices
        imgs = torch.squeeze(torch.stack(imgs))  # Shape: [16, 1, H, W]
        masks = torch.squeeze(torch.stack(masks))  # Shape: [16, 1, H, W]

        return imgs, masks


paths = glob.glob("/kaggle/input/dicom-images/*_image.nii")
ids=[]
for path in paths:
    ids.append(extract_number_from_path(path))
# Convert string IDs to integers for sorting
ids = [int(i) for i in ids]
ids.sort()

# Split into train, val, test
test_ids = ids[-20:]           # Last 20 for testing
val_ids = ids[-30:-20]         # 10 before test for validation
train_ids = ids[:-30]          # Remaining for training

# Convert back to strings if needed
test_ids = [str(i) for i in test_ids]
val_ids = [str(i) for i in val_ids]
train_ids = [str(i) for i in train_ids]

print("Train:", train_ids)
print("Validation:", val_ids)
print("Test:", test_ids)


Train: ['2', '3', '4', '5', '6', '7', '8', '12', '13', '14', '22', '26', '27', '29', '31', '34', '36', '38', '40', '41', '42', '43', '44', '45', '47', '50', '51', '52', '58', '63', '66', '67', '68', '69', '70', '72', '73', '74', '75', '76', '77', '78', '79', '80', '82', '83', '85', '86', '90', '91', '93', '98', '99', '101', '102', '103', '104', '106', '107', '108', '109', '110', '112', '113', '114', '115', '117', '122', '125', '126']
Validation: ['127', '129', '130', '133', '136', '137', '139', '143', '145', '150']
Test: ['151', '152', '154', '155', '157', '159', '160', '161', '162', '163', '164', '165', '173', '175', '180', '181', '186', '189', '191', '192']


In [5]:
pip install segmentation_models_pytorch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 3.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:

import segmentation_models_pytorch as smp
model = smp.Unet(
    encoder_name="resnet50",        # choose encoder, e.g. mobilenet_v2 or efficientnet-b7
    encoder_weights="imagenet",     # use `imagenet` pre-trained weights for encoder initialization
    in_channels=1,                  # model input channels (1 for gray-scale images, 3 for RGB, etc.)
    classes=1,                      # model output channels (number of classes in your dataset)
)
import torch
model.load_state_dict(torch.load('/kaggle/input/retina-segmentation-by-pytorch-f1-score-80/checkpoints/best.pth'),strict=False)

config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/102M [00:00<?, ?B/s]

/tmp/ipykernel_19/3890943007.py:9: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('/kaggle/input/retina-segmentation-by-pytorch-f1-score-80/c

_IncompatibleKeys(missing_keys=['encoder.conv1.weight'], unexpected_keys=[])

In [7]:
    train_dataset = VolumeSliceDataset("/kaggle/input/dicom-images/", "/kaggle/input/dicom-labels/",train_ids, target_shape=(128, 128))
    val_dataset = VolumeSliceDataset("/kaggle/input/dicom-images/", "/kaggle/input/dicom-labels/",val_ids, target_shape=(128, 128))
    test_dataset = VolumeSliceDataset("/kaggle/input/dicom-images/", "/kaggle/input/dicom-labels/",test_ids, target_shape=(128, 128))
    
    train_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=16)
    val_dataloader = DataLoader(val_dataset, batch_size=1, shuffle=True, num_workers=16)
    test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=16)

    # model = UNet()
    import torch
    
    def dice_score(preds, targets, threshold=0.5):
        preds = (preds > threshold).float()
        targets = targets.float()
    
        intersection = (preds * targets).sum(dim=(1,2,3))
        union = preds.sum(dim=(1,2,3)) + targets.sum(dim=(1,2,3))
        dice = (2. * intersection + 1e-8) / (union + 1e-8)
    
        return dice.mean().item()
    
    def iou_score(preds, targets, threshold=0.5):
        preds = (preds > threshold).float()
        targets = targets.float()
    
        intersection = (preds * targets).sum(dim=(1,2,3))
        union = (preds + targets).clamp(0,1).sum(dim=(1,2,3)) - intersection
        iou = (intersection + 1e-8) / (union + 1e-8)
    
        return iou.mean().item()


    
    # Initialize metrics
    # dice = BinaryDice()
    # iou = BinaryJaccardIndex()
    # preds and target should be of shape (N, ...) and type float or bool
    # dice_score = dice(preds, target)
    # iou_score = iou(preds, target)

    def evaluate(model, dataloader, device,best_dice):
        model.eval()
        model.to(device)
        epoch_iou_score=0
        total_dice, total_iou = 0.0, 0.0
        with torch.no_grad():
            for img, mask in dataloader:
                img, mask = img.to(device), mask.to(device)
                img, mask = img.view(-1, 1, 128, 128), mask.view(-1, 1, 128, 128)
    
                out = model(img)
                out = torch.sigmoid(out)  # Ensure outputs are in [0, 1] for binary mask
                y = mask.round().long()
                tp, fp, fn, tn = smp.metrics.get_stats(out, y, mode='binary', threshold=0.5)
                iou_score = smp.metrics.iou_score(tp, fp, fn, tn, reduction="micro").item()
                # print(f'val_acc: {acc}')
                epoch_iou_score += iou_score
            # size += y.shape[0]
        return epoch_iou_score/len(dataloader)

    loss_fn = smp.losses.DiceLoss(mode="binary")

    train(model, train_dataloader,val_dataloader,test_dataloader, loss_fn,epochs=100)

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 16 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
Epoch 1/100: 100%|██████████| 70/70 [00:54<00:00,  1.28batch/s, loss=0.442]

Epoch 1: Loss = 0.6451


VALIDATION IoU = 0.4617773413658142


Epoch 2/100: 100%|██████████| 70/70 [00:52<00:00,  1.32batch/s, loss=0.212]

Epoch 2: Loss = 0.3306


...

Epoch 100/100: 100%|██████████| 70/70 [00:43<00:00,  1.61batch/s, loss=0.0411]

Epoch 100: Loss = 0.0281


VALIDATION IoU = 0.8461697578430176


/tmp/ipykernel_19/4258048742.py:178: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best.pth'))


TEST IoU = 0.8664802879095077
